### 네이버 영화 리뷰 감성분석 2
- tokenizer 함수
- TfidfVectorizer + NaiveBayes

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('../data/네이버영화리뷰_train_전처리완료.tsv', sep='\t')
test_df = pd.read_csv('../data/네이버영화리뷰_test_전처리완료.tsv', sep='\t')

In [3]:
X_train = train_df.document.values
y_train = train_df.label.values
X_test = test_df.document.values
y_test = test_df.label.values

##### 1. Tokenizer 함수

In [4]:
with open('../data/한글불용어 copy.txt', encoding='utf-8') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]

In [5]:
from konlpy.tag import Okt
okt = Okt()

In [6]:
def okt_tokenizer(text):
    morphs = okt.morphs(text, stem=True)
    tokens = [word for word in morphs if word not in stop_words]
    return tokens

##### 2. Pipeline 학습

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression


In [8]:
pipeline = Pipeline([
    ('tvect', TfidfVectorizer(tokenizer=okt_tokenizer)),
    ('nb', MultinomialNB())
])
%time pipeline.fit(X_train, y_train)

CPU times: total: 5min 53s
Wall time: 5min 41s


Pipeline(steps=[('tvect',
                 TfidfVectorizer(tokenizer=<function okt_tokenizer at 0x000002A90D36EEF0>)),
                ('nb', MultinomialNB())])

In [9]:
%time pipeline.score(X_test, y_test)

CPU times: total: 1min 50s
Wall time: 1min 51s


0.8356464423155654

##### 3. 실제 데이터 적용

In [10]:
reviews = [
    '이 영화 개꿀잼 ㅋㅋㅋ',
    '이 영화 핵노잼 ㅠㅠ', 
    '이딴게 영화냐 ㅉㅉ',
    '감독 뭐하는 놈이냐?',
    '와 개쩐다 정말 세계관 최강자들의 영화다'
]


In [11]:
import re

reviews = list(map(lambda s: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', s), reviews))
reviews

['이 영화 개꿀잼 ㅋㅋㅋ',
 '이 영화 핵노잼 ㅠㅠ',
 '이딴게 영화냐 ㅉㅉ',
 '감독 뭐하는 놈이냐 ',
 '와 개쩐다 정말 세계관 최강자들의 영화다']

In [12]:
pipeline.predict(reviews)

array([1, 0, 0, 0, 1], dtype=int64)

##### 최적 파라메터 찾기
- 매 시행마다 한글 형태소 분석을 하느라 시간이 많이 소요됨
- 최적 파라메터를 찾으려면 형태소 분석이 끝난 데이터로 

In [16]:
# CountVectorizer에서 찾은 최적의 파라메터를 TfidfVectorizer에 적용
pipeline2 = Pipeline([
    ('tvect', TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1, 3), max_df=0.91)),
    ('nb', MultinomialNB())
])
%time pipeline2.fit(X_train, y_train)

CPU times: total: 7min 42s
Wall time: 7min 44s


Pipeline(steps=[('tvect',
                 TfidfVectorizer(max_df=0.91, ngram_range=(1, 3),
                                 tokenizer=<function okt_tokenizer at 0x000002A90D36EEF0>)),
                ('nb', MultinomialNB())])

In [17]:
%time pipeline2.score(X_test, y_test)

CPU times: total: 2min 5s
Wall time: 2min 5s


0.8626668304265946

In [ ]:
pipeline2.predict()

In [8]:
# TfidfVectorizer와  LogisticRegression 적용
from sklearn.model_selection import GridSearchCV
pipeline3 = Pipeline([
    ('tvect', TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1, 3),  max_df=0.91)),
    ('lr', LogisticRegression(random_state=2023, max_iter=500))
])

params = {
    'lr__C' : [0.01, 0.1, 1, 5, 10]
}
grid_pipe = GridSearchCV(pipeline3, params, scoring='accuracy', cv=3)

In [9]:
%time grid_pipe.fit(X_train, y_train)

Wall time: 2h 36min 52s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tvect',
                                        TfidfVectorizer(max_df=0.91,
                                                        ngram_range=(1, 3),
                                                        tokenizer=<function okt_tokenizer at 0x000001A25C51D160>)),
                                       ('lr',
                                        LogisticRegression(max_iter=500,
                                                           random_state=2023))]),
             param_grid={'lr__C': [0.01, 0.1, 1, 5, 10]}, scoring='accuracy')

In [10]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.8633832801113568